In [1]:
%autosave 120

Autosaving every 120 seconds


In [32]:
import pandas as pd
#import spacy
#pip install nltk
from nltk.tokenize import word_tokenize
import numpy as np
#import en_core_web_sm
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import mutual_info_classif, f_classif
from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from pprint import pprint
from time import time
from sklearn.linear_model import SGDClassifier
from sklearn import metrics


# Load Data

In [16]:
comments = pd.read_csv("data/comments_preprocessed_final.csv")
comments_5k = comments.iloc[:5000,]
comments_5k.head(5)

,id,comment_text,target
0,59848,this be so cool -PRON- be like ' would -PRON- ...,0
1,59849,thank -PRON- this would make -PRON- life a lot...,0
2,59852,this be such an urgent design problem kudo to ...,0
3,59855,be this something i will be able to install on...,0
4,59856,haha -PRON- guy be a bunch of loser,1


# Naive Bayes

In [45]:
#X = comments_5k.comment_text
#y = comments_5k.target
X = comments.comment_text
y = comments.target
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state = 17)

In [24]:
# use count vectorizer
# partly retreived from https://www.kaggle.com/catris25/multinomial-naive-bayes-with-countvectorizer

vectorizer = CountVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_valid = vectorizer.transform(X_valid)

mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred = mnb.predict(X_valid)

print("Accuracy:",metrics.accuracy_score(y_valid, y_pred))


Accuracy: 0.910564238160442


In [38]:
# use tf-idf
# retreived from https://iq.opengenus.org/naive-bayes-on-tf-idf-vectorized-matrix/

tf_vectorizer = TfidfVectorizer()

X_train_tf = tf_vectorizer.fit_transform(X_train)
X_valid_tf = tf_vectorizer.transform(X_valid)
print("X_train has n_samples: %d, n_features: %d" % X_train_tf.shape)
print("X_valid has n_samples: %d, n_features: %d" % X_valid_tf.shape)

mnb = MultinomialNB()
mnb.fit(X_train_tf, y_train)
y_pred = mnb.predict(X_valid_tf)

score = metrics.accuracy_score(y_valid, y_pred)
print("accuracy:   %0.3f" % score)

print(metrics.classification_report(y_valid, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_valid, y_pred))

print('------------------------------')

X_train has n_samples: 1235441, n_features: 198843
X_valid has n_samples: 529475, n_features: 198843
accuracy:   0.921
              precision    recall  f1-score   support

           0       0.92      1.00      0.96    487019
           1       0.89      0.02      0.04     42456

    accuracy                           0.92    529475
   macro avg       0.91      0.51      0.50    529475
weighted avg       0.92      0.92      0.89    529475

confusion matrix:
[[486914    105]
 [ 41587    869]]
------------------------------


In [30]:
# Cross Validation for count vectorizer
X_cv = vectorizer.fit_transform(X)
scores_5fd = cross_val_score(mnb, X_cv, y, cv=5)
print("Cross validated scores:", scores_5fd)
print("Mean cross validated scores:", scores_5fd.mean())

Cross validated scores: [0.90394749 0.90314831 0.90901545 0.89723868 0.90089891]
Mean cross validated scores: 0.9028497667323991


In [46]:
# Cross Validation for tfidf

tf_vectorizer = TfidfVectorizer() 

X_tf = tf_vectorizer.fit_transform(X)
#X_valid_tf = tf_vectorizer.transform(X_valid)
print("X_train has n_samples: %d, n_features: %d" % X_tf.shape)
#print("X_valid has n_samples: %d, n_features: %d" % X_valid_tf.shape)

mnb = MultinomialNB()

scores_5fd = cross_val_score(mnb, X_tf, y, cv=5)
print("Cross validated scores:", scores_5fd)
print("Mean cross validated scores:", scores_5fd.mean())

X_train has n_samples: 1764916, n_features: 238205
Cross validated scores: [0.92109557 0.92072989 0.92061657 0.92088854 0.92093954]
Mean cross validated scores: 0.9208540235107222


# SVM 

In [ ]:
'''
    cited from scikit-learn...
    Using pipeline and grid search to tune parameters of vectorization, feature selection and model fitting. 
'''
pipeline = Pipeline([
                    ('vect', CountVectorizer(stop_words='english')), 
                    ('kbest', SelectKBest()), 
    # why mutual_info_classif, not f_classifier: https://medium.com/@hertan06/which-features-to-use-in-your-model-350630a1e31c
                     ('clf', SGDClassifier(random_state=100)), 
                    ])
parameters={
    'vect__ngram_range': ((1, 1), (1, 2)), 
    'kbest__score_func': (mutual_info_classif, chi2, f_classif),
    'kbest__k': (1000,2000,'all'),
    #'clf__alpha': [1e-7, 1e0],
    #'clf__alpha': [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0],
    # C / alpha overfits or not.
    'clf__penalty': ('l2', 'l1'), #l2: ridge regression
    'clf__loss': ('hinge', 'log') #hinge (Default): a linear SVM, log: logistic regression
    
}

if __name__ == '__main__':
    grid_search=GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=1)
    
    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)
    t0 = time()
    
    grid_search.fit(X_train_tidy, y_train)
    
    print("done in %0.3fs" % (time() - t0))
    print()
    
    results = grid_search.cv_results_
    test_scores = results['mean_test_score']
    params = results['params']
    rank = results['rank_test_score']
    
    print("best score: %0.3f" % grid_search.best_score_)
    best_parameters = grid_search.best_estimator_.get_params()
    print("Best parameters set: ")
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

    print("Top 5 best parameter sets based on accuracy score: ")
    list(np.take(params,list(rank.argsort()[:5])))
    

In [ ]:
# the parameter sets that gives the highest accuracy score
grid_search.best_params_

In [ ]:
# the top three parameter sets have similar accuracy
np.take(test_scores, list(rank.argsort()[:5]))

In [ ]:
results = grid_search.cv_results_
test_scores = results['mean_test_score']
params = results['params']
rank = results['rank_test_score']
print("Top 5 best parameter sets based on accuracy score: ")
list(np.take(params,list(rank.argsort()[:5])))

In [ ]:
# calcuate train & test accuracy using tuned parameters
vectorizer = CountVectorizer(stop_words='english')
X_train_tidy_vec = vectorizer.fit_transform(X_train_tidy)

vocab_train1, vocab_val1, y_vocab_train1, y_vocab_val1 = train_test_split(X_train_tidy_vec, y_train, test_size=0.3, random_state = 100)

fs = SelectKBest(score_func=mutual_info_classif, k=1000)
fs_train = fs.fit(vocab_train1, y_vocab_train)
transformer_train = fs.transform(vocab_train1)
transformer_val = fs.transform(vocab_val1)

sgd_clf = SGDClassifier(loss='log', penalty='l1').fit(transformer_train, y_vocab_train1)
print("Accuracy after parameter tunning: ")
print("train accuracy: ", cross_val_score(sgd_clf, transformer_train, y_vocab_train1).mean())
print("test accuracy: ", cross_val_score(sgd_clf, transformer_val, y_vocab_val1).mean())

In [ ]:
'''
    Source: scikit-learn... , 
    Validation curve shows the accuracy score of using different regularization parameter values
'''
from sklearn.model_selection import validation_curve, learning_curve
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
train_scores, test_scores = validation_curve(SGDClassifier(loss='log', penalty='l1'), 
                                             X_train_tidy_vec, y_train, "alpha", 
                                             np.logspace(-15, 0, 50),
                                             cv = 5, n_jobs=-1, verbose=10)

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

param_range = np.logspace(-15, 0, 50)
plt.title("Validation Curve for Linear Regression")
plt.xlabel("Log alpha value")
plt.ylabel("Accuracy Score")
plt.ylim(0.8, 1.05)
lw = 2

plt.semilogx(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1,
                 color="darkorange", lw=lw)
plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1,
                 color="navy", lw=lw)
plt.legend(loc="best")
plt.show()

# finding the value C with the highest validation accuracy score
best_alpha = param_range[test_scores_mean.argmax()]
print("best log alpha value:", best_alpha)

In [ ]:
'''
    Code cited from scikit-learn/Plotting Learning Curves... 
    Learning curve is plotted to show how well a model can learn from the data. 
    Closer the validation score is to the training score, better the model is. 
'''
train_size, learn_train, learn_test = learning_curve(SGDClassifier(loss='log', penalty='l1', alpha = best_alpha),  
                                                    X_train_tidy_vec, y_train, 
                                                    train_sizes = np.linspace(0.00001,1,50), 
                                                    cv = 5, n_jobs=-1, verbose=10)
mean_learn_train = np.mean(learn_train, axis=1)
std_learn_train = np.std(learn_train, axis=1)
mean_learn_test = np.mean(learn_test, axis=1)
std_learn_test = np.std(learn_test, axis=1)

train_sizes = np.linspace(0.000001,1,50)
plt.title("Learning Curve for Linear Regression")
plt.xlabel("training examples")
plt.ylabel("Accuracy Score")
plt.ylim(0.9, 1.05)
lw = 2

plt.plot(train_sizes, mean_learn_train, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(train_sizes, mean_learn_train - std_learn_train,
                 mean_learn_train + std_learn_train, alpha=0.1,
                 color="darkorange", lw=lw)
plt.semilogx(train_sizes, mean_learn_test, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(train_sizes, mean_learn_test - std_learn_test,
                 mean_learn_test + std_learn_test, alpha=0.1,
                 color="navy", lw=lw)
plt.legend(loc="best")
plt.show()

In [ ]:
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, 
    grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))

